# Model manager of the Load Curve modelling 

This notebook is intented to be a user-friendly approach of the **load curve model**. Just run the steps you need. 
## 1 - Make dataset 
This steps is the whole preprocessing pipeline of the data. From the download of weather data, the preparation of annual demands, hourly load data to the split into train set and test set. Inputs to provide during the run : **split_week_end** (e.g whether you want to split week end in saturday and sunday) , **not_use_wem_inputs** (whether you want to use WEM inputs hourly load by subsector for the training. If False, the model uses the annual demand of subsector only. There is just one phase of training.), **pop_weighted** (e.g whether you want the WEM inputs hourly_load disaggregation using population averaged or using annual demand)

## 2 - Train model

This is the train model step. The model architecture is the following : **The model estimates the hourly demand of end-use cluster from the total load electricity demand and explanatory variables such as temperature, solar irradiance and time features**

The model is composed of a **MultiLayer perceptron for each end-use-cluster**. These MLP are trained together in order to compute the total load and compare it to the actual load. **The inputs are explanatory variables** (temperature, solar irradiance, features) and **the annual demand for the end-use cluster for each country.** How it works ? The artifical neural network finds correlation between explanatory variables and the total load to fit the actual load. 

## 3 - Predict results

Basically predict results on the test set and assess the performance of the model. After running that cell. You can find here : **Load_curve_modelling/models/Results analysis.pynb** the whole routine to visualize graphs using the outputs of the model and save them + evaluate model performance.

# 1 - Make dataset 

Run the following cell if datasets are not already computed and saved to disk. 

In [ ]:
#if kernel died for no reason run this cell: 
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True' 

In [ ]:
%run src/data/make_dataset.py 

### 1.1 Let's have a quick look to the data set

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.max_columns",None)
df = pd.read_csv('data/processed/processed_historical_test_set.csv')


ax1 = sns.relplot(x='hour_of_day', y='temperature',hue = 'month', kind='line', 
            data=df,palette = 'coolwarm',legend = 'full',facet_kws=dict(sharey=False),ci = None)
ax1.fig.subplots_adjust(top=0.93)
ax1.fig.suptitle('Temperature', fontsize = 20, fontweight = 'bold')
ax1.set(ylim = (0))

ax2 = sns.relplot(x='hour_of_day', y='irradiance_surface',hue = 'month', kind='line', 
            data=df,palette = 'coolwarm',legend = 'full',facet_kws=dict(sharey=False),ci = None)
ax2.fig.subplots_adjust(top=0.93)
ax2.fig.suptitle('Irradiance level', fontsize = 20, fontweight = 'bold')
ax2.set(ylim = (0))

plt.figure()
sns.lineplot(x='hour_of_day', y='services_rate', 
            data=df,label = 'services_rate')
sns.lineplot(x='hour_of_day', y='occupation_rate', 
            data=df,label = 'occupation_rate')
sns.lineplot(x='hour_of_day', y='activity_rate', 
            data=df,label = 'activity_rate')
plt.legend(bbox_to_anchor=(1.05, 1),borderaxespad=0)
plt.title('Time dependant rate',fontsize = 20, fontweight = 'bold')
plt.ylim(0)

df.country.unique().tolist()

# 2 - Train model

The trained models are saved here : **Load_curve_modelling/models/version_number** Note that you have a model trained on all countries and one folder per country. It denotes all the country specific models.

### 2.1 Define the parameters of the model

If you make changes, run the cells to update the parameters files.
You can find here for each subsectors the **features associated** (e.g explanatory variables used by the model to find correlation between them and the subsector's load), and sometimes a **penalty** with other **hyper parameters**. 

#### 2.1.1 Irradiance penalty definition

It aims to produce curves not only based on the irradiance penalty but also on the activty of services & residential. This loss aims to **add a penalty when the irradiance surface is higher than a certain threshold and when people are not active/awaken**. Because lighting is unlikely to happen when the irradiance surface is high/people are not active.  It is defined as followed : 
#### $loss = [f_{lighting}(t)(\alpha p_{hour} + p_{lighting})]^{2}$  
##### where :  $p_{hour} = 1 - activity$
##### and : $p_{lighting} = \max(0,irradiance(t) - threshold) $

#### 2.1.1 Temperature penalty definition

This loss aims to **add a penalty when the temperature is higher/lower than a certain temperature threshold because cooling/heating is unlikely to happen when the temperature is low/high**.  It is defined as followed : 
#### $loss = \alpha (f_{heating/cooling}(t)\Delta T) ^2 $  
##### where :  $\Delta T = \max (0,T(t) - threshold)$ *if heating*
##### and : $\Delta T = \max (0,threshold-T(t))$ *if cooling*

In [ ]:
import yaml 

#The subsector mapping first : 
subsector_mapping = {
        'clu_01': {
            'subsectors': [
                'RES_CK', 'SER_OT', 'RES_AP',
                'IND_CE', 'IND_IS', 'IND_NS', 'IND_CH', 'IND_AL', 'IND_PA',
                'IND_MI', 'IND_CO', 'IND_TR', 'IND_MA', 'IND_FO', 'IND_WO',
                'IND_TE', 'TRA_RA', 'TRA_RO',
            ],
            'features': ['cos_h', 'sin_h', 'weekday', 'cos_w', 'sin_w','services_rate'],
        },
        'res_LI': {
            'subsectors': ['RES_LI'],
            'features': ['cos_h', 'sin_h', 'weekday', 'irradiance_surface','activity_rate'],
            'irradiance_penalty_res' : {'threshold': 350, 'weight' : 5} #weight corresponds to the alpha term (see above)
            },
        'ser_LI': {
            'subsectors': ['SER_LI'],
            'features': ['cos_h', 'sin_h', 'weekday', 'irradiance_surface','services_rate'],
            'irradiance_penalty_ser' : {'threshold': 350, 'weight' :1 } #weight corresponds to the alpha term (see above)
            },
        'res_SH': {
            'subsectors': ['RES_SH', 'RES_WH'],
            'features': ['cos_h', 'sin_h', 'weekday', 'temperature','occupation_rate'],
            'heating_penalty': {'threshold': 18.,
                                'weight': 1} #weight corresponds to the alpha term (see above)
        },
        'ser_SH': {
            'subsectors': ['SER_SH', 'SER_WH'],
            'features': ['cos_h', 'sin_h', 'weekday', 'temperature','services_rate'],
            'heating_penalty': {'threshold': 18.,
                                'weight': 1} #weight corresponds to the alpha term (see above)
        },
        'res_SC': {
            'subsectors': ['RES_SC'],
            'features': ['cos_h', 'sin_h', 'weekday', 'temperature', 'cos_w', 'sin_w','occupation_rate'],
            'cooling_penalty': {'threshold': 20.,
                                'weight': 0.1} #weight corresponds to the alpha term (see above)
        },
        'ser_SC': {
            'subsectors': ['SER_SC'],
            'features': ['cos_h', 'sin_h', 'weekday', 'temperature', 'cos_w', 'sin_w','services_rate'],
            'cooling_penalty': {'threshold': 20.,
                                'weight': 0.1} #weight corresponds to the alpha term (see above)
        }
    }
#The confidence interval percentage of each cluster for WEM training
ci_ratio = {'CLU_01' : 0.3, 'CLU_LI' : 0.2 , 'CLU_SH' : 0.2 , 'CLU_SC' : 0.2}

#The parameters of the neural network 
hparams = {'n_layers' : 1, #Number of layers in the LSTM (has to be done manually for the MLP)
        'max_epochs' : 400, 
        'epochs_wem' : 0.15, # Percentage to apply on max_epochs to compute the max_epochs_wem
        'batch_size':4,
        'dim_hidden': 40, #Number of neurons in the hidden layer of the MLP
        #'dim_hidden_lstm' : 2, #Number of hidden nodes in the LSTM
        'optimizer_name': 'adam', #Optimizer of the ANN
        'dropout_rate' : 0,
        'lr': 0.001, #learning rate of the ANN
        'subsector_mapping': subsector_mapping,
        'not_use_wem_inputs' : True, #Don't touch this argument   
        'h_ratio' : ci_ratio
    }

#Saving these parameters as configuration files : 
with open('models/logs/hparams_init.yaml','w') as yamlfile : 
    yaml.dump(hparams,yamlfile)
with open('models/logs/subsector_mapping.yaml','w') as yamlfile : 
    yaml.dump(subsector_mapping,yamlfile)
with open('models/logs/ci_ratio.yaml','w') as yamlfile : 
    yaml.dump(ci_ratio,yamlfile)

### 2.2 Run the model

In [ ]:
%run src/models/train_model.py

#if kernel died for no reason try : 
#import os 
#os.environ['KMP_DUPLICATE_LIB_OK']='True' 

## 3 - Predict model

The prediction of the models are saved here : **Load_curve_modelling/models/version_number in excel format**. Then visualize and assess model performance using the ***Results analysis*** notebook

In [ ]:
#Get the last version number of the directory
import os
list = os.listdir('models/logs')
maximum = 'version_0'
for l in list : 
    if 'version' in l : 
        if int(l.split(sep = '_')[1]) > int(maximum.split(sep = '_')[1]) : 
            maximum = l
       
print('Last version in the directory is',maximum)

%run src/models/predict_model.py

## 4 - Connect the data of the load curve model with the World Energy Model

The output is an excel file with the same format as in the WEM excel files here : *G:\EO2021\Model\Sectors\RES\DSR\Model*.

In [ ]:
%run src/data/connect_wem.py